In [221]:
from collections import defaultdict, Counter
import re

import pandas as pd
from tqdm.auto import tqdm

In [222]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../..")
from berp.languages import english

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [223]:
input_path = "../../cmudict-0.7b"
output_path = "cmudict_ipa.csv"

In [224]:
brackets_re = re.compile(r"\(\d+\)")
stress_re = re.compile(r"[012]")
comments_re = re.compile(r"\s*#.+$")

In [225]:
mapping = defaultdict(list)

In [226]:
for word, pron in english.cmudict_overrides.items():
    mapping[word].append(pron)

In [227]:
with open(input_path, encoding="latin-1") as f:
    i = 0
    for line in tqdm(f.readlines()):
        line = line.strip()
        if line.startswith(";;;"):
            continue

        word, arpa = line.split(" ", 1)
        word = brackets_re.sub("", word)
        arpa = stress_re.sub("", arpa).strip().split(" ")
        
        result = []
        for part in arpa:
            ipa = english.cmu_ipa_mapping[part]
            result.append(ipa)
            
        mapping[word].append(" ".join(result))

  0%|          | 0/134429 [00:00<?, ?it/s]

In [228]:
# Prepare dataframe
rows = [(word.lower(), i, pron) for word, prons in mapping.items()
        for i, pron in enumerate(prons)]
df = pd.DataFrame(rows, columns=["word", "pronunciation_idx", "pronunciation"])
df

,word,pronunciation_idx,pronunciation
0,was,0,w ʌ z
1,was,1,w ɑ z
2,was,2,w ɑ z
3,wind,0,w ɪ n d
4,wind,1,w aɪ n d
...,...,...,...
134415,{brace,0,b ɹ ɛɪ s
134416,{left-brace,0,l ɛ f t b ɹ ɛɪ s
134417,{open-brace,0,oʊ p ɛ n b ɹ ɛɪ s
134418,}close-brace,0,k l oʊ z b ɹ ɛɪ s


In [229]:
df.to_csv(output_path, index=False)